In [1]:
import sys, os, time, pickle
from timeit import default_timer as timer
from humanfriendly import format_timespan

In [2]:
import pandas as pd
import numpy as np

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [26]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.metrics import classification_report

In [3]:
from dotenv import load_dotenv
load_dotenv('admin.env')

True

In [4]:
from db_connect_mag import Session, Paper, PaperAuthorAffiliation, db

In [5]:
# test_papers_df = pd.read_pickle('data/collect_haystack_20180409/test_papers.pickle')
# target_papers_df = pd.read_pickle('data/collect_haystack_20180409/target_papers.pickle')
# train_papers_df = pd.read_pickle('data/collect_haystack_20180409/train_papers.pickle')

In [6]:
# this is the data for the fortunato review on Community Detection in Graphs
start = timer()
test_papers_df = pd.read_pickle('data/collect_haystack_20180409_2/test_papers.pickle')
target_papers_df = pd.read_pickle('data/collect_haystack_20180409_2/target_papers.pickle')
train_papers_df = pd.read_pickle('data/collect_haystack_20180409_2/train_papers.pickle')
print(format_timespan(timer()-start))

10.88 seconds


In [7]:
with open('data/collect_haystack_20180409_2/counter.pickle', 'rb') as f:
    c = pickle.load(f)

In [8]:
def get_target_in_test(test, target, id_colname='Paper_ID'):
    return set.intersection(set(test[id_colname]), set(target[id_colname]))
len(get_target_in_test(test_papers_df, target_papers_df))

397

In [9]:
len(target_papers_df)

397

In [14]:
test_subset = test_papers_df.sample(n=100000)

In [15]:
len(get_target_in_test(test_subset, target_papers_df))

18

In [16]:
# remove the train (seed) papers from the test set (haystack)
n_before = len(test_subset)
test_subset = test_subset.drop(train_papers_df.index, errors='ignore')
n_after = len(test_subset)
print("removed {} seed papers from the haystack. size of haystack: {}".format(n_before-n_after, n_after))

removed 0 seed papers from the haystack. size of haystack: 100000


In [17]:
start = timer()
target_ids = set(target_papers_df.Paper_ID)
test_papers_df['target'] = test_subset.Paper_ID.apply(lambda x: x in target_ids)
print(format_timespan(timer()-start))

0.52 seconds


In [19]:
# def tree_distance(n1, n2, sep=":"):
#     # https://en.wikipedia.org/wiki/Lowest_common_ancestor
#     # the distance from v to w can be computed as 
#     # the distance from the root to v, plus the distance from 
#     # the root to w, minus twice the distance from 
#     # the root to their lowest common ancestor
#     v, w = [n.split(sep) for n in [n1, n2]]
#     distance_root_to_v = len(v)
#     distance_root_to_w = len(w)
    
#     distance_root_to_lca = 0
#     for i in range(min(distance_root_to_v, distance_root_to_w)):
#         if v[i] == w[i]:
#             distance_root_to_lca += 1
#         else:
#             break
#     return distance_root_to_v + distance_root_to_w - (2*distance_root_to_lca)

In [20]:
def tree_distance(n1, n2, sep=":"):
    # since depth is sort of arbitrary, let's try this
    v, w = [n.split(sep) for n in [n1, n2]]
    distance_root_to_v = len(v)
    distance_root_to_w = len(w)
    avg_depth = (distance_root_to_v + distance_root_to_w) * .5
    
    distance_root_to_lca = 0
    for i in range(min(distance_root_to_v, distance_root_to_w)):
        if v[i] == w[i]:
            distance_root_to_lca += 1
        else:
            break
    return (avg_depth - distance_root_to_lca) / avg_depth

In [21]:
def avg_distance(cl, cl_group):
    distances = []
    for x in cl_group:
        distances.append(tree_distance(cl, x))
    return sum(distances) / len(distances)

In [22]:
n_before = len(test_subset)
test_subset = test_subset.dropna(subset=['title'])
n_after = len(test_subset)
print("dropped {} rows".format(n_before-n_after))

dropped 6605 rows


In [82]:
test_subset = test_subset.reset_index()

In [83]:
start = timer()
vect = CountVectorizer()
data = test_subset.title.append(train_papers_df.title).tolist()
vect.fit(data)

print(format_timespan(timer()-start))

1.33 second


In [84]:
start = timer()
tf_train = vect.transform(train_papers_df.title.tolist())
print(format_timespan(timer()-start))

0 seconds


In [85]:
start = timer()
tf_test = vect.transform(test_subset.title.tolist())
print(format_timespan(timer()-start))

1.16 second


In [86]:
start = timer()
tf_global = vect.transform(data)
print(format_timespan(timer()-start))

1.66 second


In [87]:
tf_transform = TfidfTransformer()
tf_transform.fit(tf_global)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [88]:
tfidf_train = tf_transform.transform(tf_train)

In [90]:
tfidf_test = tf_transform.transform(tf_test)

In [92]:
tfidf_test.shape

(93395, 50876)

In [94]:
from sklearn.metrics.pairwise import cosine_similarity

In [95]:
csims = cosine_similarity(tfidf_test, tfidf_train.mean(axis=0))

In [97]:
test_subset = test_subset.join(pd.Series(csims.flatten(), name='title_tfidf_cosine_similarity'))

In [99]:
test_subset.sort_values('title_tfidf_cosine_similarity', ascending=False)

,index,EF,Paper_ID,cl,title,year,title_tfidf_cosine_similarity
59445,1171939,7.707870e-09,2092102472,3372652:1:1:1281,detection of community structure in networks b...,2012.0,0.583353
3435,107272,1.992670e-06,2095293504,3372652:1:1:2,finding and evaluating community structure in ...,2004.0,0.582090
38142,166307,5.605830e-09,2483140568,1115862:1,community detection in social networks,2015.0,0.528806
89978,1528215,5.434740e-09,2554784184,3372652:1:1:2817,evolutionary community detection in complex an...,2016.0,0.522203
39834,79044,1.703530e-08,2032721088,902576:3:10,o r in the community,1981.0,0.516389
57222,1268234,6.230730e-09,2053229448,3372652:1:1:1613,finding community structure in spatially const...,2015.0,0.513658
86563,123169,1.621630e-08,2026143132,22916:2:2:9,adaptive clustering algorithm for community de...,2008.0,0.513363
50236,1599825,7.191750e-09,2014541072,3372652:1:1:665,detecting the community structure in complex n...,2008.0,0.493491
24901,2541308,5.525310e-09,2616094075,3372652:1:1:2732,adaptive community detection in complex networ...,2017.0,0.480367
30605,144134,6.579060e-08,125376580,3372652:1:1:65,an algorithm to find overlapping community str...,2007.0,0.470975


In [30]:
start = timer()
test_papers_df['avg_distance_to_train'] = test_papers_df.cl.apply(avg_distance, cl_group=train_papers_df.cl.tolist())
print(format_timespan(timer()-start))

4 minutes and 0.95 seconds


In [31]:
test_papers_df.sort_values(['avg_distance_to_train', 'EF'], ascending=[True, False]).head(50)

,EF,Paper_ID,cl,title,year,target,avg_distance_to_train
107272,1.992670e-06,2095293504,3372652:1:1:2,finding and evaluating community structure in ...,2004.0,False,0.611111
110154,9.501080e-07,2131681506,3372652:1:1:9,fast unfolding of communities in large networks,2008.0,False,0.611111
109495,8.579680e-07,2120043163,3372652:1:1:7,comparing community structure identification,2005.0,False,0.611111
114759,3.484730e-07,2606584716,3372652:1:1:29,e mail as spectroscopy automated discovery of ...,2005.0,False,0.611111
110902,8.711380e-08,2139818818,3372652:1:1:55,mixture models and exploratory analysis in net...,2007.0,False,0.611111
107228,8.128820e-08,2091202730,3372652:1:1:52,detect overlapping and hierarchical community ...,2009.0,False,0.611111
109443,6.873940e-08,2117526408,3372652:1:1:68,towards real time community detection in large...,2009.0,False,0.611111
118641,2.728230e-08,1967752035,3372652:1:1:148,finding instabilities in the community structu...,2005.0,False,0.611111
123223,2.550930e-08,2033507223,3372652:1:1:128,quantifying and identifying the overlapping co...,2009.0,False,0.611111
106490,2.393100e-08,2069629462,3372652:1:1:170,comparison and validation of community structu...,2006.0,False,0.611111


In [32]:
test_papers_df.groupby('target')['EF', 'avg_distance_to_train'].describe().T

target                              False         True 
EF                    count  2.612894e+06  3.970000e+02
                      mean   3.942335e-08  9.342797e-07
                      std    3.210841e-07  2.630972e-06
                      min    5.434740e-09  7.338580e-09
                      25%    5.717760e-09  2.819180e-08
                      50%    7.594130e-09  9.191400e-08
                      75%    1.667440e-08  4.857660e-07
                      max    1.716360e-04  2.707530e-05
avg_distance_to_train count  2.612894e+06  3.970000e+02
                      mean   9.971833e-01  8.228729e-01
                      std    2.735958e-02  1.772978e-01
                      min    6.111111e-01  6.161111e-01
                      25%    1.000000e+00  6.161111e-01
                      50%    1.000000e+00  8.505556e-01
                      75%    1.000000e+00  1.000000e+00
                      max    1.000000e+00  1.000000e+00

In [33]:
import matplotlib.pyplot as plt

In [34]:
%matplotlib inline

In [37]:
# http://scikit-learn.org/stable/auto_examples/hetero_feature_union.html
class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    ItemSelector is not designed to handle data grouped by sample.  (e.g. a
    list of dicts).  If your data is structured this way, consider a
    transformer along the lines of `sklearn.feature_extraction.DictVectorizer`.

    Parameters
    ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

In [100]:
class ClusterTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, colname='cl'):
        self.colname = colname
        
    def fit(self, x, y=None):
        return self
    
    def transform(self, df):
        avg_dist = df[self.colname].apply(avg_distance, cl_group=train_papers_df.cl.tolist())
        return avg_dist.as_matrix().reshape(-1, 1)

In [101]:
class DataFrameColumnTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, colname):
        self.colname = colname
        
    def fit(self, x, y=None):
        return self
    
    def transform(self, df):
        return df[self.colname].as_matrix().reshape(-1, 1)

In [190]:
pipeline = Pipeline([
    ('union', FeatureUnion(
        transformer_list = [
            ('avg_distance_to_train', Pipeline([
#                 ('selector', ItemSelector(key='avg_distance_to_train')),
#                 ('vect', DictVectorizer(X.avg_distance_to_train.to_dict))
                ('cl_feat', ClusterTransformer()),
            ])),
            ('ef', Pipeline([
#                 ('selector', ItemSelector(key='avg_distance_to_train')),
#                 ('vect', DictVectorizer(X.avg_distance_to_train.to_dict))
                ('ef_feat', DataFrameColumnTransformer('EF')),
            ])),
            
            # NOTE: this is just to test.
            # we probably want features that relate the titles to the seed papers. not just straight features in test set.
            ('title_bow', Pipeline([
                ('selector', ItemSelector(key='title')),
                ('tfidf', TfidfVectorizer(min_df=10)),
            ]))
        ],
    )),
    
    ('svc', SVC(kernel='linear', probability=True))
])

In [191]:
# X = test_papers_df[['EF', 'avg_distance_to_train']]
X = test_papers_df[test_papers_df.title.notnull()]
# Fortunato paper was published in 2010
X = X[X.year<=2010]

# y = test_papers_df['target']
y = X['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=999)

In [192]:
start = timer()
pipeline.fit(X_train, y_train)
print(format_timespan(timer()-start))

1 hour, 41 minutes and 34.3 seconds


In [193]:
start = timer()
# y_pred_proba = model.predict_proba(X_test)[:, 1]
y_pred_proba = pipeline.predict_proba(X)[:, 1]
print(format_timespan(timer()-start))
y_pred_proba


6 minutes and 32.97 seconds


array([2.52947189e-05, 4.21541038e-05, 7.42309923e-05, ...,
       1.28252908e-04, 1.77193432e-04, 1.38100486e-04])

In [194]:
y_pred_proba.shape

(1521097,)

In [195]:
pred_ranks = pd.Series(y_pred_proba, index=X.index, name='pred_ranks')
test_papers_df.join(pred_ranks).sort_values('pred_ranks', ascending=False).head()

,EF,Paper_ID,cl,title,year,target,avg_distance_to_train,pred_ranks
110985,2.130870e-07,2146591355,3372652:1:1:30,community structure in large networks natural ...,2009.0,True,0.616111,0.929936
114140,1.096330e-08,2171551254,3372652:1:1:624,analysis of bidding networks in ebay aggregate...,2007.0,True,0.616111,0.893195
123194,2.714580e-08,2029373408,3372652:1:1:140,efficient modularity optimization by multistep...,2008.0,True,0.616111,0.873199
107817,1.115010e-08,2103980282,3372652:1:1:376,revealing network communities through modulari...,2009.0,True,0.616111,0.861771
119424,1.272820e-08,1971844861,3372652:1:1:292,multistep greedy algorithm identifies communit...,2008.0,True,0.616111,0.842784


In [196]:
len(test_papers_df)

2613291

In [197]:
len(X)

1521097

In [198]:
# top_predictions = test_papers_df.join(pred_ranks).sort_values('pred_ranks', ascending=False).head(len(target_papers_df))
top_predictions = X.join(pred_ranks).sort_values('pred_ranks', ascending=False).head(len(target_papers_df))

In [199]:
top_predictions.groupby('target')['Paper_ID'].count()

target
False    213
True     184
Name: Paper_ID, dtype: int64

In [200]:
top_predictions.pred_ranks.min()

0.017612516717949805

In [201]:
start = timer()
y_test_pred = pipeline.predict(X_test)
print(format_timespan(timer()-start))

1 minute and 12.06 seconds


In [202]:
print(classification_report(y_test, y_test_pred))

             precision    recall  f1-score   support

      False       1.00      1.00      1.00    304138
       True       0.00      0.00      0.00        82

avg / total       1.00      1.00      1.00    304220



/home/jporteno/code/autoreview/venv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [159]:
pipeline = Pipeline([
    ('union', FeatureUnion(
        transformer_list = [
            ('avg_distance_to_train', Pipeline([
#                 ('selector', ItemSelector(key='avg_distance_to_train')),
#                 ('vect', DictVectorizer(X.avg_distance_to_train.to_dict))
                ('cl_feat', ClusterTransformer()),
            ])),
            ('ef', Pipeline([
#                 ('selector', ItemSelector(key='avg_distance_to_train')),
#                 ('vect', DictVectorizer(X.avg_distance_to_train.to_dict))
                ('ef_feat', DataFrameColumnTransformer('EF')),
            ])),
            
            # NOTE: this is just to test.
            # we probably want features that relate the titles to the seed papers. not just straight features in test set.
#             ('title_bow', Pipeline([
#                 ('selector', ItemSelector(key='title')),
#                 ('tfidf', TfidfVectorizer(min_df=10)),
#             ]))
        ],
    )),
    
    ('logreg', LogisticRegression())
])

In [160]:
# X = test_papers_df[['EF', 'avg_distance_to_train']]
X = test_papers_df[test_papers_df.title.notnull()]
# Fortunato paper was published in 2010
X = X[X.year<=2010]

# y = test_papers_df['target']
y = X['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=999)

In [161]:
start = timer()
pipeline.fit(X_train, y_train)
print(format_timespan(timer()-start))

1 minute and 59.51 seconds


In [162]:
start = timer()
# y_pred_proba = model.predict_proba(X_test)[:, 1]
y_pred_proba = pipeline.predict_proba(X)[:, 1]
print(format_timespan(timer()-start))
y_pred_proba


2 minutes and 21.45 seconds


array([0.00016925, 0.00016925, 0.00016925, ..., 0.00016925, 0.00016925,
       0.00016925])

In [163]:
y_pred_proba.shape

(1521097,)

In [164]:
pred_ranks = pd.Series(y_pred_proba, index=X.index, name='pred_ranks')
test_papers_df.join(pred_ranks).sort_values('pred_ranks', ascending=False).head()

,EF,Paper_ID,cl,title,year,target,avg_distance_to_train,pred_ranks
107272,1.992670e-06,2095293504,3372652:1:1:2,finding and evaluating community structure in ...,2004.0,False,0.611111,0.062686
110154,9.501080e-07,2131681506,3372652:1:1:9,fast unfolding of communities in large networks,2008.0,False,0.611111,0.062686
109495,8.579680e-07,2120043163,3372652:1:1:7,comparing community structure identification,2005.0,False,0.611111,0.062686
114759,3.484730e-07,2606584716,3372652:1:1:29,e mail as spectroscopy automated discovery of ...,2005.0,False,0.611111,0.062686
110902,8.711380e-08,2139818818,3372652:1:1:55,mixture models and exploratory analysis in net...,2007.0,False,0.611111,0.062686


In [165]:
len(test_papers_df)

2613291

In [166]:
len(X)

1521097

In [167]:
# top_predictions = test_papers_df.join(pred_ranks).sort_values('pred_ranks', ascending=False).head(len(target_papers_df))
top_predictions = X.join(pred_ranks).sort_values('pred_ranks', ascending=False).head(len(target_papers_df))

In [168]:
top_predictions.groupby('target')['Paper_ID'].count()

target
False    270
True     127
Name: Paper_ID, dtype: int64

In [169]:
top_predictions.pred_ranks.min()

0.058318146270451454

In [170]:
start = timer()
y_test_pred = pipeline.predict(X_test)
print(format_timespan(timer()-start))

28.94 seconds


In [171]:
print(classification_report(y_test, y_test_pred))

             precision    recall  f1-score   support

      False       1.00      1.00      1.00    304138
       True       0.00      0.00      0.00        82

avg / total       1.00      1.00      1.00    304220



/home/jporteno/code/autoreview/venv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [32]:
# what if we only use pagerank?
X = test_papers_df[['EF']]
y = test_papers_df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=999)

start = timer()
_model = LogisticRegression()
_model.fit(X_train, y_train)
print(format_timespan(timer()-start))

# y_pred_proba = model.predict_proba(X_test)[:, 1]
_y_pred_proba = _model.predict_proba(X)[:, 1]
#y_pred_proba

print(y_pred_proba.shape)

_pred_ranks = pd.Series(_y_pred_proba, index=X.index, name='pred_ranks')
#test_papers_df.join(_pred_ranks).sort_values('pred_ranks', ascending=False).head()



_top_predictions = test_papers_df.join(_pred_ranks).sort_values('pred_ranks', ascending=False).head(len(target_papers_df))

_top_predictions.groupby('target')['Paper_ID'].count()

4.23 seconds
(2613291,)


target
False    388
True       9
Name: Paper_ID, dtype: int64

In [33]:
# what if we only use avg distance?
X = test_papers_df[['avg_distance_to_train']]
y = test_papers_df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=999)

start = timer()
_model = LogisticRegression()
_model.fit(X_train, y_train)
print(format_timespan(timer()-start))

# y_pred_proba = model.predict_proba(X_test)[:, 1]
_y_pred_proba = _model.predict_proba(X)[:, 1]
#y_pred_proba

print(y_pred_proba.shape)

_pred_ranks = pd.Series(_y_pred_proba, index=X.index, name='pred_ranks')
#test_papers_df.join(_pred_ranks).sort_values('pred_ranks', ascending=False).head()



_top_predictions = test_papers_df.join(_pred_ranks).sort_values('pred_ranks', ascending=False).head(len(target_papers_df))

_top_predictions.groupby('target')['Paper_ID'].count()

4.6 seconds
(2613291,)


target
False    377
True      20
Name: Paper_ID, dtype: int64

In [34]:
start = timer()
toplevels = test_papers_df.cl.apply(lambda x: x.split(":")[0])
print(format_timespan(timer()-start))

2.39 seconds


In [55]:
toplevels.name = 'toplevel'

In [37]:
toplevels_set = set(toplevels)

In [46]:
start = timer()
tbl = db.tables['clusters_meta_tree']
sq = tbl.select(tbl.c.toplevel_in_tree.in_(toplevels_set))
# r = db.engine.execute(sq).fetchall()
cl_meta = db.read_sql(sq)
print(format_timespan(timer()-start))

/home/jporteno/code/autoreview/venv/lib/python3.5/site-packages/pymysql/cursors.py:165: Warning: (3170, "Memory capacity of 8388608 bytes for 'range_optimizer_max_mem_size' exceeded. Range optimization was not done for this query.")
  result = self._query(query)


19.19 seconds


In [50]:
cl_meta = cl_meta.set_index('id')

In [82]:
train_papers_df['toplevel'] = train_papers_df.cl.apply(lambda x: x.split(":")[0]).astype(int)

In [83]:
meta_map = cl_meta.set_index('toplevel_in_tree').meta_cl

In [84]:
train_papers_df['cl_meta'] = train_papers_df.toplevel.map(meta_map)

In [87]:
test_papers_df['toplevel'] = toplevels.astype(int)
test_papers_df['cl_meta'] = test_papers_df.toplevel.map(meta_map)

In [89]:
start = timer()
test_papers_df['meta_avg_distance_to_train'] = test_papers_df.cl_meta.apply(avg_distance, cl_group=train_papers_df.cl_meta.tolist())
print(format_timespan(timer()-start))

4 minutes and 10.75 seconds


In [94]:
# logistic regression including meta cl
X = test_papers_df[['EF', 'avg_distance_to_train', 'meta_avg_distance_to_train']]
y = test_papers_df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=999)

start = timer()
model_meta = LogisticRegression()
model_meta.fit(X_train, y_train)
print(format_timespan(timer()-start))

# y_pred_proba = model.predict_proba(X_test)[:, 1]
y_pred_proba_meta = model_meta.predict_proba(X)[:, 1]
#y_pred_proba

print(y_pred_proba_meta.shape)

pred_ranks_meta = pd.Series(y_pred_proba_meta, index=X.index, name='pred_ranks')
#test_papers_df.join(_pred_ranks).sort_values('pred_ranks', ascending=False).head()



top_predictions_meta = test_papers_df.join(pred_ranks_meta).sort_values('pred_ranks', ascending=False).head(len(target_papers_df))

top_predictions_meta.groupby('target')['Paper_ID'].count()

6.54 seconds
(2613291,)


target
False    289
True     108
Name: Paper_ID, dtype: int64

In [105]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y, y_pred_proba))
print(roc_auc_score(y, y_pred_proba_meta))
print(roc_auc_score(y, _y_pred_proba))

0.9553407108497369
0.8686914172329787
0.7952530679672806
